In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np 
import math 
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter 
import scipy
from matplotlib import cm as CM
import torch
%matplotlib inline

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

: 

: 

In [8]:
root = './ShanghaiTech/'
part_A_train = os.path.join(root,'part_A/train_data','images')
part_A_train_mat = os.path.join(root,'part_A/train_data','ground-truth')
part_A_test = os.path.join(root,'part_A/test_data','images')
part_A_test_mat = os.path.join(root,'part_A/test_data','ground-truth')

In [10]:
import re
import os
import pandas as pd
import fnmatch
from torchvision.io import read_image
from torchvision.io import ImageReadMode
import csv

dictCache = {}

class ImageResultsDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, actual_dir, UNet_dir, YOLO_dir, transform=None):
        self.img_dir = img_dir
        self.UNet_dir = UNet_dir
        self.YOLO_dir = YOLO_dir
        self.transform = transform

        with open(UNet_dir,newline='') as file:
          reader = csv.reader(file)
          self.UNet_dict = dict(reader)  # pull in each row as a key-value pair
          
        with open(YOLO_dir,newline='') as file:
          reader = csv.reader(file)
          self.YOLO_dict = dict(reader)  # pull in each row as a key-value pair

        with open(actual_dir,newline='') as file:
          reader = csv.reader(file)
          self.actual_dict = dict(reader)  # pull in each row as a key-value pair

    def __len__(self):
        return len(fnmatch.filter(os.listdir(self.img_dir), '*.*'))

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f"IMG_{idx+1}.jpg")
        image = read_image(img_path, mode=ImageReadMode.RGB)
        image = image.to(dtype = torch.float)
        
        res = img_path.split("/")
        res = res[-1].split("\\")
        
        if res[0] == 'train_data':
          unetdictquery = "images_train/" + f"IMG_{idx+1}.jpg"
          yoloquery ="images_train/" + f"IMG_{idx+1}.jpg"
          actualquery ="images_train/" + f"IMG_{idx+1}.jpg"
        elif res[0] == 'test_data':
          unetdictquery = "images_test/" + f"IMG_{idx+1}.jpg"
          yoloquery = "images_test/" + f"IMG_{idx+1}.jpg"
          actualquery = "images_test/" + f"IMG_{idx+1}.jpg"

        UNet_count = float(self.UNet_dict[unetdictquery])
        YOLO_count = int(self.YOLO_dict[yoloquery])
        actual_count = float(self.actual_dict[actualquery])

        if self.transform:
            image = self.transform(image)
        return image, UNet_count, YOLO_count, actual_count

In [11]:
actual_dir_train = "./FinalModelCounts/personActual_train.txt" 
UNet_dir_train = "./FinalModelCounts/personUNet_train.txt" # 
YOLO_dir_train = "./FinalModelCounts/personYOLO_train.txt"

In [12]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import multiprocessing as mp
import numpy as np
import random
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np 
import math 

use_from_total = 1
seed_ = 41
use_val = False
batch_size = 1
transform = transforms.Compose([
                                transforms.CenterCrop(448),
                                transforms.Normalize(mean=(0,)*3, std=(255,)*3)
                              ]) 


#* load image and mask folders, print class dict
print("Loading Dataset...")
dataset = ImageResultsDataset(img_dir=part_A_train, actual_dir = actual_dir_train, 
                UNet_dir=UNet_dir_train, YOLO_dir = YOLO_dir_train, transform=transform)


Loading Dataset...


In [13]:
for i in range(len(dataset)):
  print("|", end="")

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [14]:
from torchvision import datasets, models
def CNN_DenseNet(pretrained=True):
    model = models.densenet121(
        weights="IMAGENET1K_V1"
    )  # Densenet121 model pretrained on ImageNet1K
    num_ftrs = model.classifier.in_features  # densenet output
    print(num_ftrs)
    model.classifier = nn.Linear(num_ftrs, 2)  # fully connected output to binary classifier
    model = model.to(device)  # send to gpu
    return model

In [15]:
learning_rate = 0.0001 
weight_decay_ = 0.001

model = CNN_DenseNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay_)

1024


In [16]:
print("Training Starting...")
batch_size = 16

#* dataloader generator with set seed
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed_)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=0, pin_memory=True, shuffle=True, worker_init_fn=seed_worker, generator=g)

num_epochs = 10
total_step = len(train_loader)

for epoch in range(num_epochs):
  last_loss = 0
  print("\nStarting epoch:", epoch+1)
  len_loss = 0
  for i, (image, UNet_count, YOLO_count, actual_count) in enumerate(train_loader):
    UNet_count /= 3 # testing
    print(UNet_count)
    print(YOLO_count)
    print(actual_count)
    binary_labs = torch.zeros(2, len(actual_count))
    print(binary_labs.shape)
    for c in range(len(actual_count)):
      dist_YOLO = (abs(YOLO_count[c] - actual_count[c]))
      dist_UNet = (abs(UNet_count[c] - actual_count[c]))
      if dist_YOLO <= dist_UNet:
        binary_labs[1][c] = 1 # yolo
      else:
        binary_labs[0][c] = 1 # unet
    binary_labs.to(device)
    print(binary_labs)

    image = image.to(device)
    #* forward
    output = model(image)
    loss   = loss_fn(output, binary_labs)
    
    #* change the params
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    #* add up loss
    last_loss += (loss.item())
    len_loss += 1

    if (i+1) % 100 == 0 or i == 0:
            print ('\nEpoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                  .format(epoch+1, num_epochs, i+1, total_step, (last_loss)/(len_loss)))
    print("|", end ="")

    del image
    del UNet_count
    del YOLO_count
    del actual_count
    del binary_labs
    del output
    del loss
    torch.cuda.empty_cache()
    
    break
  break

Training Starting...

Starting epoch: 1
tensor([ 19.8847, 125.9468,  29.0211,  11.9583,  57.2374, 143.1420, 116.1442,
         56.7552, 211.8562, 303.4009,  38.8044,  14.3065,  13.1098,  12.7486,
         63.6320,  57.2212], dtype=torch.float64)
tensor([18, 28, 20, 18,  8, 20, 20, 18, 27,  6, 16, 19, 22, 23, 20, 10])
tensor([ 59.6542, 377.8404,  87.0634,  35.8749, 171.7122, 429.4260, 348.4326,
        170.2657, 635.5687, 910.2026, 116.4133,  42.9194,  39.3294,  38.2459,
        190.8961, 171.6635], dtype=torch.float64)
torch.Size([2, 16])
tensor([[1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.]])


OutOfMemoryError: CUDA out of memory. Tried to allocate 196.00 MiB (GPU 0; 4.00 GiB total capacity; 3.18 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF